In [9]:
# we need to install couple libraries in order to run this code
# please open terminal and install these libraries
# pip install cdsapi 
# pip install xarray netcdf4 
# pip install dask 
import os

home_dir = os.path.expanduser("~")
with open(os.path.join(home_dir, ".cdsapirc"), "w") as f:
    f.write('url: https://cds.climate.copernicus.eu/api/v2\n')
    f.write('key: 200733:630986af-80e7-4ee4-b43e-f1899a03ba7d\n')  # replace with your actual UID and API key

In [10]:
import cdsapi
import numpy as np
import pandas as pd
import csv
import xarray as xr
import dask

c = cdsapi.Client()

# Define the latitude/longitude boundaries for each state.
states = {
    'Alabama': [30.137521, -88.473228, 34.984, -84.889103],
    'Florida': [24.396308, -87.634643, 31.000888, -80.031362],
    'Georgia': [30.357851, -85.605165, 35.000659, -80.751429],
    'Kentucky': [36.497129, -89.571509, 39.147732, -81.964970],
    'Mississippi': [30.1476, -91.655009, 34.996052, -88.097889],
    'North Carolina': [33.752877, -84.321869, 36.588117, -75.400120],
    'South Carolina': [32.0346, -83.3539, 35.215402, -78.540047],
    'Tennessee': [34.9829, -90.310298, 36.6781, -81.6469],
}
# Initialize a dictionary to hold the average temperatures for each state and each week of each year
state_temperatures = {state: {} for state in states}

# Define the years of interest
years = ["2016", "2017", "2018", "2019", "2020"]

# Use Dask to handle computations that don't fit into memory
dask.config.set(scheduler='processes')

# Iterate through each state and its boundaries
for state, boundaries in states.items():
    north, west, south, east = boundaries

    # Iterate through each year of interest
    for year in years:
        # Request the data from the CDS
        # Save the data as a netCDF file named "{state}_{year}.nc"
        c.retrieve(
            "reanalysis-era5-land",
            {
                "variable": "2m_temperature",
                "year": year,
                "month": ["01", "02", "03", "04", "05", "06", "07", "08", "09", "10", "11", "12"],
                "day": ["01", "02", "03", "04", "05", "06", "07", "08", "09", "10", "11", "12", "13", "14", "15", "16", "17", "18", "19", "20", "21", "22", "23", "24", "25", "26", "27", "28", "29", "30", "31"],
                "time": ["00:00", "06:00", "12:00", "18:00"],
                "format": "netcdf",
                "area": [north, west, south, east],
            },
            f"{state}_{year}.nc"
        )

        # Open the netCDF file using xarray with Dask
        ds = xr.open_dataset(f"{state}_{year}.nc", chunks={})

        # Convert the xarray Dataset to a pandas DataFrame.
        # Ensure that the time variable is used as the index.
        data = ds['t2m'].to_dataframe().reset_index().set_index('time')

        # Ensure the data is sorted by date.
        data = data.sort_index()

        # Resample the data to weekly frequency, calculating the mean for each week.
        weekly_data = data.resample('W').mean()

        # Convert temperatures from Kelvin to Fahrenheit
        weekly_data['t2m'] = (weekly_data['t2m'] - 273.15) * 9/5 + 32

        # Calculate the ISO week number and month for each week and add them as columns.
        weekly_data['week'] = weekly_data.index.to_series().apply(lambda x: x.isocalendar()[1])
        weekly_data['month'] = weekly_data.index.to_series().apply(lambda x: x.month)

        # Save the weekly data to the state_temperatures dictionary.
        state_temperatures[state][year] = weekly_data

# Create the CSV file and write the headers.
with open('state_temperatures.csv', 'w', newline='') as csvfile:
    fieldnames = ['State', 'Year', 'Month', 'Week', 'AverageTemperature']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()

    # Write the data for each state, year, week and average temperature.
    for state, years in state_temperatures.items():
        for year, weeks in years.items():
            for week, row in weeks.iterrows():
                writer.writerow({'State': state, 'Year': year, 'Month': row['month'], 'Week': row['week'], 'AverageTemperature': row['t2m']})

2023-05-31 13:06:09,131 INFO Welcome to the CDS
2023-05-31 13:06:09,131 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-land
2023-05-31 13:06:09,681 INFO Request is completed
2023-05-31 13:06:09,681 INFO Downloading https://download-0015-clone.copernicus-climate.eu/cache-compute-0015/cache/data7/adaptor.mars.internal-1685490975.5304685-21280-11-293e7e75-5054-4606-80f2-bf0f9011240d.nc to Alabama_2016.nc (4.9M)
2023-05-31 13:06:12,338 INFO Download rate 1.9M/s   
2023-05-31 13:06:14,062 INFO Welcome to the CDS
2023-05-31 13:06:14,064 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-land
2023-05-31 13:06:14,321 INFO Downloading https://download-0009-clone.copernicus-climate.eu/cache-compute-0009/cache/data5/adaptor.mars.internal-1685491809.3860183-9593-20-33c12c37-b396-40dc-900f-d95e90d2b22e.nc to Alabama_2017.nc (4.9M)
2023-05-31 13:06:18,729 INFO Download rate 1.1M/s   
2023-05-31 13:06:20,516 INFO Welc